In [19]:
from pipline_extract import extract_latest_loaders

dataloaders = extract_latest_loaders()
train_loader = dataloaders['train']
val_loader = dataloaders['validation']

Pipeline artifact [: 1c535be5-fc01-42c9-b1bd-a82f30a727c8] loaded successfully


In [20]:
def get_input_example():
    batch = next(iter(train_loader))

    # Move the batch to CPU if needed (for logging purposes)
    for key in batch:
        batch[key] = batch[key].cpu()

    # Prepare the input example
    return {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"],
        "has_source": batch["has_source"]
    }

In [21]:
import torch.nn as nn
import torch

from transformers import BertModel


class SentimentAnalysisModel(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', num_labels=3):
        super(SentimentAnalysisModel, self).__init__()

        self.bert = BertModel.from_pretrained(bert_model_name)

        self.linear1 = nn.Linear(self.bert.config.hidden_size + 1, num_labels)

        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, has_source):
        embeddings = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        has_source = has_source.unsqueeze(1) 
        combined_input = torch.cat((embeddings, has_source), dim=1)

        regularized = self.dropout(combined_input)
        logits = self.linear1(regularized)

        return logits


In [22]:
from tqdm import tqdm

import mlflow
import mlflow.pytorch

def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch):
    model.train()
    train_loss = 0.0
    total = 0.

    loop = tqdm(
        enumerate(dataloader, 1),
        total=len(dataloader),
        desc=f"Epoch {epoch}: train",
        leave=True,
    )

    for _, batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        has_source = batch['has_source'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        logits = model(input_ids = input_ids, attention_mask=attention_mask, has_source=has_source)

        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item() * input_ids.size(0)
        total += labels.size(0)

        loop.set_postfix({"loss": train_loss/total})

    avg_train_loss = train_loss / total
    mlflow.log_metric('train_loss', avg_train_loss, step=epoch)


def val_one_epoch(model, dataloader, criterion, device, epoch, best_so_far, ckpt_name='model'):
    model.eval()
    val_loss = 0.
    correct = 0.
    total = 0.
    with torch.no_grad():
        loop = tqdm(
            enumerate(dataloader, 1),
            total=len(dataloader),
            desc=f"Epoch {epoch}: val",
            leave=True,
        )
        for i, batch in loop:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            has_source = batch['has_source'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids=input_ids, attention_mask=attention_mask, has_source=has_source)

            loss = criterion(logits, labels)
            val_loss += loss.item() * input_ids.size(0)

            _, preds = torch.max(logits, dim=1)
            correct += (preds == labels).sum().item()

            total += labels.size(0)

            loop.set_postfix({"loss": val_loss/total, "acc": correct / total})
        current_acc = correct / total

        avg_val_loss = val_loss / total
        mlflow.log_metric('validation_loss', avg_val_loss, step=epoch)
        mlflow.log_metric('validation_accuracy', current_acc, step=epoch)


        if current_acc > best_so_far:
            print(f"Validation accuracy improved from {best_so_far:.4f} to {current_acc:.4f}. Saving model...")
            mlflow.pytorch.log_model(model, ckpt_name)

            best_so_far = current_acc
    return best_so_far



In [24]:
import torch.optim as optim
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

epochs = 10
device = 'mps'
model_name = 'simple_sentiment_analysis_model'
lr = 2e-5

model = SentimentAnalysisModel(bert_model_name='bert-base-uncased', num_labels=3).to(device)
criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=lr)

best_so_far = 0.
mlflow.set_experiment("SentimentAnalysis")

with mlflow.start_run():
    mlflow.log_text(str(model), "model_architecture.txt")
    mlflow.log_param("learning_rate", 2e-5)
    mlflow.log_param("epochs", epochs)
    for epoch in range(epochs):
        train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)
        best_so_far = val_one_epoch(model, val_loader, criterion, device, epoch, best_so_far, model_name)

Epoch 0: val: 100%|██████████| 52/52 [00:02<00:00, 23.87it/s, loss=0.634, acc=0.747]


Validation accuracy improved from 0.0000 to 0.7468. Saving model...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:25                                                                                   │
│                                                                                                  │
│   22 │   mlflow.log_param("epochs", epochs)                                                      │
│   23 │   for epoch in range(epochs):                                                             │
│   24 │   │   train_one_epoch(model, train_loader, optimizer, criterion, device, epoch)           │
│ ❱ 25 │   │   best_so_far = val_one_epoch(model, val_loader, criterion, device, epoch, best_so    │
│   26                                                                                             │
│                                                                                                  │
│ in val_one_epoch:81                                                                              │
│                                                                                                  │
│   78 │   │                                                                                       │
│   79 │   │   if current_acc > best_so_far:                                                       │
│   80 │   │   │   print(f"Validation accuracy improved from {best_so_far:.4f} to {current_acc:    │
│ ❱ 81 │   │   │   mlflow.pytorch.log_model(model, ckpt_name, input_example=input_example)         │
│   82 │   │   │                                                                                   │
│   83 │   │   │   best_so_far = current_acc                                                       │
│   84 │   return best_so_far                                                                      │
│                                                                                                  │
│ /Users/maxmartyshov/Desktop/IU/year3/PMDL/Sentiment_Analysis_for_Financial_News/.venv/lib/python │
│ 3.12/site-packages/mlflow/pytorch/__init__.py:295 in log_model                                   │
│                                                                                                  │
│    292 │   │   PyTorch logged models                                                             │
│    293 │   """                                                                                   │
│    294 │   pickle_module = pickle_module or mlflow_pytorch_pickle_module                         │
│ ❱  295 │   return Model.log(                                                                     │
│    296 │   │   artifact_path=artifact_path,                                                      │
│    297 │   │   flavor=mlflow.pytorch,                                                            │
│    298 │   │   pytorch_model=pytorch_model,                                                      │
│                                                                                                  │
│ /Users/maxmartyshov/Desktop/IU/year3/PMDL/Sentiment_Analysis_for_Financial_News/.venv/lib/python │
│ 3.12/site-packages/mlflow/models/model.py:725 in log                                             │
│                                                                                                  │
│    722 │   │   │   mlflow_model = cls(                                                           │
│    723 │   │   │   │   artifact_path=artifact_path, run_id=run_id, metadata=metadata, resources  │
│    724 │   │   │   )                                                                             │
│ ❱  725 │   │   │   flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)       │
│    726 │   │   │   # `save_model` calls `load_model` to infer the model requirements, which may  │
│    727 │   │   │   # __pycache__ directories being created in the model directory.               │
│    728 │   │   │   for pycache in Path(local_path).rglob("_